In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import os
import random
import multiprocessing
import itertools
import functools
from joblib import dump, load
from pathlib import Path
import numpy as np
import numpy.ma as ma  
import pandas as pd
from tqdm import tqdm

import amp.data_utils.data_loader as data_loader
import amp.utils.classifier_utils as cu
from amp.config import MAX_LENGTH, MIN_LENGTH, VOCAB_PAD_SIZE, LATENT_DIM
from amp.data_utils import sequence as du_sequence
from amp.utils import generator, basic_model_serializer
from amp.utils.basic_model_serializer import load_master_model_components
from amp.utils.seed import set_seed
from amp.utils import phys_chem_propterties
from amp.inference.filtering import amino_based_filtering         

from sklearn import metrics, model_selection
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KernelDensity

from keras import layers, activations, Model
input_to_encoder = layers.Input(shape=(MAX_LENGTH,))
input_to_decoder = layers.Input(shape=(LATENT_DIM+2,))

import modlamp.descriptors
import modlamp.analysis
import modlamp.sequences

Using TensorFlow backend.


In [4]:
seed = 7
np.random.seed(seed)

In [5]:
def translate_generated_peptide(encoded_peptide):
    alphabet = list('ACDEFGHIKLMNPQRSTVWY')
    return ''.join([alphabet[el - 1] if el != 0 else "" for el in encoded_peptide.argmax(axis=1)])

def translate_peptide(encoded_peptide):
    alphabet = list('ACDEFGHIKLMNPQRSTVWY')
    return ''.join([alphabet[el-1] if el != 0 else "" for el in encoded_peptide])


In [6]:
models = [
    'HydrAMP',
    'PepCVAE',
    'Basic',
]


best_epochs = {
    'HydrAMP': 37,
    'PepCVAE': 35,
    'Basic': 15,
}
    

# Prepare data

In [7]:
random.seed(seed)
data_manager = data_loader.AMPDataManager(
    '../data/unlabelled_positive.csv',
    '../data/unlabelled_negative.csv',
    min_len=MIN_LENGTH,
    max_len=MAX_LENGTH)

amp_x, amp_y = data_manager.get_merged_data()
amp_x_train, amp_x_test, amp_y_train, amp_y_test = train_test_split(amp_x, amp_y, test_size=0.1, random_state=36)
amp_x_train, amp_x_val, amp_y_train, amp_y_val = train_test_split(amp_x_train, amp_y_train, test_size=0.2, random_state=36)

# Restrict the length
ecoli_df = pd.read_csv('../data/mic_data.csv')
mask = (ecoli_df['sequence'].str.len() <= MAX_LENGTH) & (ecoli_df['sequence'].str.len() >= MIN_LENGTH)
ecoli_df = ecoli_df.loc[mask]
mic_x = du_sequence.pad(du_sequence.to_one_hot(ecoli_df['sequence']))
mic_y = ecoli_df.value


mic_x_train, mic_x_test, mic_y_train, mic_y_test = train_test_split(mic_x, mic_y, test_size=0.1, random_state=36)
mic_x_train, mic_x_val, mic_y_train, mic_y_val = train_test_split(mic_x_train, mic_y_train, test_size=0.2, random_state=36)

new_train = pd.DataFrame()
new_train['value'] = list(mic_y_train)
new_train['sequence'] = list(mic_x_train)
active_peptides = new_train[new_train.value < 1.5]
active_peptides  = pd.concat([active_peptides] * 25)
new_train  = pd.concat([
    new_train,
    active_peptides,    
])
mic_x_train = np.array(list(new_train.sequence)).reshape(len(new_train), 25)
mic_y_train = np.array(new_train.value).reshape(len(new_train),)

#180194
uniprot_x_train = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../data/Uniprot_0_25_train.csv').Sequence)))
#22525
uniprot_x_val = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../data/Uniprot_0_25_val.csv').Sequence)))

/home/pszymczak/amp/new-tf/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/pszymczak/amp/new-tf/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
bms = basic_model_serializer.BasicModelSerializer()
amp_classifier = bms.load_model('../models/amp_classifier')
amp_classifier_model = amp_classifier()
mic_classifier = bms.load_model('../models/mic_classifier/')
mic_classifier_model = mic_classifier() 

In [9]:
#DATASET_AMP/MIC_TRAIN/VAL

amp_amp_train = amp_classifier_model.predict(amp_x_train, verbose=1, batch_size=10000).reshape(len(amp_x_train))
amp_mic_train = mic_classifier_model.predict(amp_x_train, verbose=1, batch_size=10000).reshape(len(amp_x_train))
amp_amp_val = amp_classifier_model.predict(amp_x_val, verbose=1, batch_size=10000).reshape(len(amp_x_val))
amp_mic_val = mic_classifier_model.predict(amp_x_val, verbose=1, batch_size=10000).reshape(len(amp_x_val))

mic_amp_train = amp_classifier_model.predict(mic_x_train, verbose=1, batch_size=10000).reshape(len(mic_x_train))
mic_mic_train = mic_classifier_model.predict(mic_x_train, verbose=1, batch_size=10000).reshape(len(mic_x_train))
mic_amp_val = amp_classifier_model.predict(mic_x_val, verbose=1, batch_size=10000).reshape(len(mic_x_val))
mic_mic_val = mic_classifier_model.predict(mic_x_val, verbose=1, batch_size=10000).reshape(len(mic_x_val))

uniprot_x_train = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../data/Uniprot_0_25_train.csv').Sequence)))
uniprot_x_val = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../data/Uniprot_0_25_val.csv').Sequence)))

uniprot_amp_train = amp_classifier_model.predict(uniprot_x_train, verbose=1, batch_size=10000).reshape(len(uniprot_x_train))
uniprot_mic_train = mic_classifier_model.predict(uniprot_x_train, verbose=1, batch_size=10000).reshape(len(uniprot_x_train))
uniprot_amp_val = amp_classifier_model.predict(uniprot_x_val, verbose=1, batch_size=10000).reshape(len(uniprot_x_val))
uniprot_mic_val = mic_classifier_model.predict(uniprot_x_val, verbose=1, batch_size=10000).reshape(len(uniprot_x_val))

training_generator = generator.concatenated_generator(
    uniprot_x_train,
    uniprot_amp_train,
    uniprot_mic_train,
    amp_x_train,
    amp_amp_train,
    amp_mic_train,
    mic_x_train,
    mic_amp_train,
    mic_mic_train,
    128
)

x_train_complete = np.concatenate([next(training_generator)[0][0] for _ in tqdm(range(1408))])

22524/22524 [==============================] - 0s 3us/step


100%|██████████| 1408/1408 [00:01<00:00, 1312.01it/s]


# Train PCA and generate peptides

In [10]:
def generate_unconstrained(
        n: int,
        decoder,
        amp_classifier,
        mic_classifier,
        latent_dim: int = 64,
):
    
    z = np.random.normal(size=(n, latent_dim))
    z = pca_decomposer.inverse_transform(z)
    z = np.vstack([z]*latent_dim)
    z_cond_pos = np.hstack([z, np.ones((n*latent_dim, 1)), np.ones((n*latent_dim, 1))])
    pos_decoded = decoder.predict(z_cond_pos, batch_size=1000)
    pos_decoded_argmax = pos_decoded.argmax(axis=2)
    pos_peptides = [translate_generated_peptide(peptide) for peptide in pos_decoded]
    pos_class_prediction = amp_classifier.predict(pos_decoded_argmax, batch_size=1000)
    pos_mic_prediction = mic_classifier.predict(pos_decoded_argmax, batch_size=1000)
    
    z_cond_neg = np.hstack([z, np.zeros((n*latent_dim, 1)), np.zeros((n*latent_dim, 1))])
    neg_decoded = decoder.predict(z_cond_neg, batch_size=1000)
    neg_decoded_argmax = neg_decoded.argmax(axis=2)
    neg_peptides = [translate_generated_peptide(peptide) for peptide in neg_decoded]
    neg_class_prediction = amp_classifier.predict(neg_decoded_argmax, batch_size=1000)
    neg_mic_prediction = mic_classifier.predict(neg_decoded_argmax, batch_size=1000)

    
    return {
        'pos_peptides': pos_peptides,
        'pos_class_prediction': pos_class_prediction,
        'pos_mic_prediction': pos_mic_prediction,
        'neg_peptides': neg_peptides,
        'neg_class_prediction': neg_class_prediction,
        'neg_mic_prediction': neg_mic_prediction,   
    }

In [11]:
n = 50000

for model in models:
    AMPMaster = bms.load_model(f'../models/{model}/{int(best_epochs[model])}')
    encoder_model =  AMPMaster.encoder(input_to_encoder)
    decoder_model = AMPMaster.decoder(input_to_decoder)
    # Replace Gumbel with Softmax
    if model in ['PepCVAE', 'Basic']:
        new_act = layers.TimeDistributed(
            layers.Activation(activations.softmax),
            name='decoder_time_distribute_activation')
        decoder_model.layers.pop()
        x = new_act(decoder_model.layers[-1].output)
        decoder_model = Model(input=decoder_model.input, output=[x]) 
    x_train_pred = encoder_model.predict(x_train_complete, verbose=1)
    pca_decomposer = PCA()
    pca_decomposer.fit_transform(x_train_pred)
    dump(pca_decomposer, f'../models/{model}/pca_decomposer.joblib')
    
    model_results = generate_unconstrained(
        n=n,
        decoder=decoder_model,
        amp_classifier=amp_classifier_model,
        mic_classifier=mic_classifier_model,
    )
    dump(model_results, f'../results/unconstrained_{model}.joblib')

tracking <tf.Variable 'temperature:0' shape=() dtype=float32, numpy=0.1> temperature
540672/540672 [==============================] - 277s 513us/step
tracking <tf.Variable 'temperature:0' shape=() dtype=float32, numpy=0.1> temperature
   128/540672 [..............................] - ETA: 11:01

<ipython-input-11-07ce2d5a54c8>:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  decoder_model = Model(input=decoder_model.input, output=[x])


540672/540672 [==============================] - 268s 496us/step
tracking <tf.Variable 'temperature:0' shape=() dtype=float32, numpy=0.6023884> temperature
    32/540672 [..............................] - ETA: 30:26

<ipython-input-11-07ce2d5a54c8>:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  decoder_model = Model(input=decoder_model.input, output=[x])


540672/540672 [==============================] - 263s 487us/step


In [12]:
hydra_results = load(f'../results/unconstrained_{models[0]}.joblib')
pepcvae_results = load(f'../results/unconstrained_{models[1]}.joblib')
basic_results = load(f'../results/unconstrained_{models[2]}.joblib')

In [13]:
def select_peptides(results):
    peptides = np.array(results['pos_peptides']).reshape(64, -1).T
    amp = (results['pos_class_prediction'] < 0.8).reshape(64, -1)
    mic = results['pos_mic_prediction'].reshape(64, -1)
    combined = ma.masked_where(amp, mic)
    good = combined.argmax(axis=0)
    good_peptides = peptides[list(range(peptides.shape[0])), good]
    good_amp = np.array(results['pos_class_prediction']).reshape(64, -1).T[list(range(peptides.shape[0])), good]
    good_mic = np.array(results['pos_mic_prediction']).reshape(64, -1).T[list(range(peptides.shape[0])), good]
    return pd.DataFrame.from_dict({
        'sequence': good_peptides.tolist(), 
        'amp': good_amp.tolist(),
        'mic': good_mic.tolist(),
    }
    )

In [14]:
hydra_pd = select_peptides(hydra_results)
pepcvae_pd = select_peptides(pepcvae_results)
basic_pd = select_peptides(basic_results)

In [15]:
def final_filtering(dataset):
    dataset = dataset[(dataset['amp'] > 0.8) & (dataset['mic'] > 0.8)]
    dataset = amino_based_filtering('../data/unlabelled_positive.csv', dataset)
    return dataset

In [16]:
hydra_clean = final_filtering(hydra_pd).reset_index()

In [17]:
phys_chem = pd.DataFrame.from_dict(phys_chem_propterties.calculate_physchem_prop(hydra_clean.sequence.tolist()))
phys_chem['sequence'] = hydra_clean.sequence

In [18]:
hydra_phys = pd.merge(hydra_clean, phys_chem, on='sequence').drop(['index'], axis=1)
hydra_phys = hydra_phys[hydra_phys['h_score'] > 1.5]
hydra_final = hydra_phys.sort_values('h_score', ascending=False).reset_index(drop=True)

In [ ]:
hydra_final.to_csv('../results/hydra_unconstrained.csv')